
---

# Transformaciones
---


In [1]:
import pandas as pd
import ast
df = pd.read_csv('C:\\Users\\fede\\Desktop\\LABs 1\\Proyecto individual - Federico Piparo\\Movies\\movies_dataset.csv', low_memory=False)

In [2]:
# Contar el número de filas duplicadas
duplicados = df.duplicated().sum()

# Eliminar filas duplicadas
df_sin_duplicados = df.drop_duplicates()

# Almacenar el número de filas duplicadas en una variable
numero_de_duplicados = duplicados

# Mostrar el DataFrame sin duplicados
print(df_sin_duplicados)
print("Número de filas duplicadas eliminadas:", numero_de_duplicados)


       adult                              belongs_to_collection    budget  \
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1      False                                                NaN  65000000   
2      False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3      False                                                NaN  16000000   
4      False  {'id': 96871, 'name': 'Father of the Bride Col...         0   
...      ...                                                ...       ...   
45461  False                                                NaN         0   
45462  False                                                NaN         0   
45463  False                                                NaN         0   
45464  False                                                NaN         0   
45465  False                                                NaN         0   

                                                  genres  \
0      [{'id': 

In [55]:

'''Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.'''
df = df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])


In [56]:
'''Los valores nulos del campo release date deben eliminarse.'''

df = df.dropna(subset=['release_date'])

In [57]:
'''Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.'''

df[['revenue', 'budget']] = df[['revenue', 'budget']].fillna(0)

In [58]:
'''De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.'''

#convertimos las fechas al datatype datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce') 
df['release_year'] = df['release_date'].dt.year

In [59]:
'''Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget,
cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.'''
# Convertir las columnas 'revenue' y 'budget' a tipo numérico
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
# Rellenar valores nulos en 'revenue' y 'budget' con 0
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

# Crear la columna 'return' calculando revenue / budget
# Usar np.where para evitar división por 0
import numpy as np
df['return'] = np.where(df['budget'] == 0, 0, df['revenue'] / df['budget'])

In [60]:
df['return'].head(5)

0    12.451801
1     4.043035
2     0.000000
3     5.090760
4     0.000000
Name: return, dtype: float64

In [61]:
''' Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una
lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API!
O bien buscar la manera de acceder a esos datos sin desanidarlos.'''
# Función para convertir cadenas a diccionarios o listas
def convertir_en_obj(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except (ValueError, SyntaxError):
            return x  # Retorna la cadena si no es un diccionario válido
    return x

# Aplicar la conversión a las series 
df['belongs_to_collection'] = df['belongs_to_collection'].apply(convertir_en_obj)
df['genres'] = df['genres'].apply(convertir_en_obj)
df['production_companies'] = df['production_companies'].apply(convertir_en_obj)
df['production_countries'] = df['production_countries'].apply(convertir_en_obj)
df['spoken_languages'] = df['spoken_languages'].apply(convertir_en_obj)

# Desanidar diccionarios
BTC_df = df['belongs_to_collection'].apply(pd.Series)
BTC_df = BTC_df.drop(columns = 0, errors='ignore')  # Eliminar columna de índice si existe
nuevos_nombres_BTC = ['BTCid', 'BTCname', 'BTCposter_path', 'BTCbackdrop_path']
BTC_df.columns = nuevos_nombres_BTC

# Desanidar listas
genres_df = df['genres'].apply(pd.Series)
production_companies_df = df['production_companies'].apply(pd.Series)
production_countries_df = df['production_countries'].apply(pd.Series)
spoken_languages_df = df['spoken_languages'].apply(pd.Series)

# Asegurarse de que no haya nombres de columnas duplicados
genres_df = genres_df.rename(columns=lambda x: f'genre_{x}')
production_companies_df = production_companies_df.rename(columns=lambda x: f'company_{x}')
production_countries_df = production_countries_df.rename(columns=lambda x: f'country_{x}')
spoken_languages_df = spoken_languages_df.rename(columns=lambda x: f'language_{x}')

# Eliminar las columnas originales
df = df.drop(columns=['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages'])

# Unir los DataFrames desanidado al DataFrame original
df = df.join(BTC_df)
df = df.join(production_companies_df)
df = df.join(production_countries_df)
df = df.join(spoken_languages_df)


C:\Users\fede\AppData\Local\Temp\ipykernel_11164\1652706673.py:21: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  BTC_df = df['belongs_to_collection'].apply(pd.Series)
C:\Users\fede\AppData\Local\Temp\ipykernel_11164\1652706673.py:27: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  genres_df = df['genres'].apply(pd.Series)
C:\Users\fede\AppData\Local\Temp\ipykernel_11164\1652706673.py:28: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  production_companies_df = df['production_companies'].apply(pd.Series)
C:\Users\fede\AppData\Local\Temp\ipykernel_11164\1652706673.py:29: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series


---

### Exportación de los Datafames
---


In [62]:
df.to_csv('transformados.csv', index=False)


In [63]:
BTC_df.to_csv('belongs_to_collection', index=False)

In [64]:
production_companies_df.loc[3786]

company_0               {'name': 'Fine Line Features', 'id': 8}
company_1         {'name': 'Zentropa Entertainments', 'id': 76}
company_2            {'name': 'Danmarks Radio (DR)', 'id': 119}
company_3                      {'name': 'SVT Drama', 'id': 157}
company_4                           {'name': 'Arte', 'id': 201}
company_5                    {'name': 'Memfis Film', 'id': 321}
company_6                {'name': 'France 3 Cinéma', 'id': 591}
company_7                   {'name': 'Angel films', 'id': 2996}
company_8                       {'name': 'TV 1000', 'id': 4524}
company_9                        {'name': 'Canal+', 'id': 5358}
company_10    {'name': 'Constantin Film Produktion', 'id': 5...
company_11             {'name': 'Yleisradio (YLE)', 'id': 5975}
company_12           {'name': 'Arte France Cinéma', 'id': 6916}
company_13    {'name': 'Westdeutscher Rundfunk (WDR)', 'id':...
company_14       {'name': 'Blind Spot Pictures Oy', 'id': 7330}
company_15    {'name': 'Vrijzinnig Prote